In [1]:
!pip install neattext


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
pip install torch


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install transformers


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
import joblib
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import os

In [6]:
df = pd.read_csv("emotion_dataset_raw.csv")

In [7]:
import neattext.functions as nfx
df['Clean_Text'] = df['Text'].apply(nfx.remove_userhandles)
df['Clean_Text'] = df['Clean_Text'].apply(nfx.remove_stopwords)

In [8]:
Xfeatures = df['Clean_Text']
ylabels = df['Emotion']
x_train, x_test, y_train, y_test = train_test_split(Xfeatures, ylabels, test_size=0.3, random_state=42)


In [9]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [10]:
max_seq_length = 128

def tokenize_texts(texts, max_length):
    input_ids = []
    attention_masks = []
    for text in texts:
        encoded_dict = tokenizer.encode_plus(
                            text,
                            add_special_tokens=True,
                            max_length=max_length,
                            padding='max_length',
                            truncation=True,
                            return_attention_mask=True,
                            return_tensors='pt'
                       )
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    return input_ids, attention_masks

x_train_ids, x_train_masks = tokenize_texts(x_train, max_seq_length)
x_test_ids, x_test_masks = tokenize_texts(x_test, max_seq_length)

In [11]:
class BERTbiLSTMModel(nn.Module):
    def __init__(self, bert_model, hidden_size, num_classes):
        super(BERTbiLSTMModel, self).__init__()
        self.bert = bert_model
        self.dropout = nn.Dropout(0.1)
        self.bilstm = nn.LSTM(input_size=bert_model.config.hidden_size,
                              hidden_size=hidden_size,
                              num_layers=1,
                              batch_first=True,
                              bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        bilstm_output, _ = self.bilstm(pooled_output.unsqueeze(1))  # Add an additional dimension
        logits = self.fc(bilstm_output[:, -1, :])  # Take the last hidden state
        return logits


bert_model = BertModel.from_pretrained('bert-base-uncased')
num_classes = len(df['Emotion'].unique())
hidden_size = 128
model = BERTbiLSTMModel(bert_model, hidden_size, num_classes)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

# Convert emotion labels to numerical format
emotion_to_label = {emotion: label for label, emotion in enumerate(df['Emotion'].unique())}
y_train_tensor = torch.tensor([emotion_to_label[emotion] for emotion in y_train.values])
y_test_tensor = torch.tensor([emotion_to_label[emotion] for emotion in y_test.values])

batch_size = 32
train_dataset = TensorDataset(x_train_ids, x_train_masks, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = TensorDataset(x_test_ids, x_test_masks, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

num_epochs = 15
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for input_ids, attention_mask, labels in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}'):
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        logits = model(input_ids, attention_mask)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_train_loss = total_loss / len(train_loader)
    print(f'Average training loss: {avg_train_loss:.4f}')

torch.save(model.state_dict(), 'emotion_classifier_bert_bilstm.pt')


Epoch 1/15:  72%|███████▏  | 548/762 [04:29<01:44,  2.05it/s]

In [ ]:
import torch
from transformers import BertTokenizer, BertModel
import torch.nn as nn

# Load pre-trained BERT model and tokenizer
bert_model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

model.load_state_dict(torch.load('/content/emotion_classifier_bert_bilstm.pt'))
model.eval()

# Function to predict emotion for a given sentence
def predict_emotion(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", max_length=128, truncation=True, padding='max_length')
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    with torch.no_grad():
        logits = model(input_ids, attention_mask)
        probabilities = torch.softmax(logits, dim=1)
        print(probabilities)
        predicted_class = torch.argmax(probabilities, dim=1).item()
    return predicted_class

# Example usage
sentence = "I am feeling annoying today"
predicted_emotion = predict_emotion(sentence)
print("Predicted Emotion:", predicted_emotion)


In [ ]:
emotion_to_label